# Text Summarising of the articles

In [1]:
!git clone "https://github.com/riyazweb/text-sumarize.git"

Cloning into 'text-sumarize'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 10 (delta 2), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (10/10), 541.48 KiB | 1.98 MiB/s, done.
Resolving deltas: 100% (2/2), done.


In [2]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip


--2025-01-15 12:28:42--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2025-01-15 12:28:42--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2025-01-15 12:28:42--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [12]:
# @title website run
!pip install pyngrok
import nltk
nltk.download('punkt_tab')
nltk.download('punkt')
nltk.download('stopwords')
!python /content/text-sumarize/app.py

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Public URL: NgrokTunnel: "https://6cf6-34-125-219-39.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app 'app'
 * Debug mode: on
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [15/Jan/2025 12:50:40] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Jan/2025 12:50:42] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [15/Jan/2025 12:50:47] "POST /summarize HTTP/1.1" 200 -
^C


In [3]:
# Importing the required Libraries
import numpy as np
import pandas as pd
import nltk
# nltk.download('punkt') # one time execution
import re
#nltk.download('stopwords') # one time execution
import matplotlib.pyplot as plt

from nltk.tokenize import sent_tokenize

from nltk.corpus import stopwords

from sklearn.metrics.pairwise import cosine_similarity

import networkx as nx

In [8]:
# Extract word vectors
word_embeddings = {}
file = open('/content/glove.6B.100d.txt', encoding='utf-8')
for line in file:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
file.close()
len(word_embeddings)

400000

In [10]:
# reading the file
df = pd.read_excel('/content/text-sumarize/TASK.xlsx')

In [11]:
df

,TEST DATASET,Unnamed: 1
0,NaN,Introduction
1,NaN,Acnesol Gel is an antibiotic that fights bacte...
2,NaN,Ambrodil Syrup is used for treating various re...
3,NaN,Augmentin 625 Duo Tablet is a penicillin-type ...
4,NaN,Azithral 500 Tablet is an antibiotic used to t...
...,...,...
996,NaN,Azapure Tablet belongs to a group of medicines...
997,NaN,Arimidex 1mg Tablet is used alone or with oth...
998,NaN,Arpimune ME 100mg Capsule is used to prevent y...
999,NaN,Amlodac CH Tablet is a combination medicine us...


In [12]:
df.columns

Index(['TEST DATASET', 'Unnamed: 1'], dtype='object')

In [13]:
df.rename(columns = {'Unnamed: 1' : 'Introduction' }, inplace=True)
# Deleting the first row
df.drop(0)

,TEST DATASET,Introduction
1,NaN,Acnesol Gel is an antibiotic that fights bacte...
2,NaN,Ambrodil Syrup is used for treating various re...
3,NaN,Augmentin 625 Duo Tablet is a penicillin-type ...
4,NaN,Azithral 500 Tablet is an antibiotic used to t...
5,NaN,Alkasol Oral Solution is a medicine used in th...
...,...,...
996,NaN,Azapure Tablet belongs to a group of medicines...
997,NaN,Arimidex 1mg Tablet is used alone or with oth...
998,NaN,Arpimune ME 100mg Capsule is used to prevent y...
999,NaN,Amlodac CH Tablet is a combination medicine us...


In [14]:
# Converting the DataFrame into a dictionary
text_dictionary = {}
for i in range(1,len(df['TEST DATASET'])):
    text_dictionary[i] = df['Introduction'][i]

print(text_dictionary[1])

Acnesol Gel is an antibiotic that fights bacteria. It is used to treat acne, which appears as spots or pimples on your face, chest or back. This medicine works by attacking the bacteria that cause these pimples.Acnesol Gel is only meant for external use and should be used as advised by your doctor. You should normally wash and dry the affected area before applying a thin layer of the medicine. It should not be applied to broken or damaged skin. Avoid any contact with your eyes, nose, or mouth. Rinse it off with water if you accidentally get it in these areas. It may take several weeks for your symptoms to improve, but you should keep using this medicine regularly. Do not stop using it as soon as your acne starts to get better. Ask your doctor when you should stop treatment.Common side effects like minor itching, burning, or redness of the skin and oily skin may be seen in some people. These are usually temporary and resolve on their own. Consult your doctor if they bother you or do not

**There are 1000 such description of the different medicines. The task is to give summarised form of these description.**

In [15]:
# function to remove stopwords
def remove_stopwords(sen):
    stop_words = stopwords.words('english')

    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

In [16]:
# function to make vectors out of the sentences
def sentence_vector_func (sentences_cleaned) :
    sentence_vector = []
    for i in sentences_cleaned:
        if len(i) != 0:
            v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
        else:
            v = np.zeros((100,))
        sentence_vector.append(v)

    return (sentence_vector)

In [17]:
# function to get the summary of the articles
# NOTE - Remove '#' infront of print statement for displaying the contents at different stages of the text summarisation process
def summary_text (test_text, n = 5):
    sentences = []

    # tokenising the text
    sentences.append(sent_tokenize(test_text))
    # print(sentences)
    sentences = [y for x in sentences for y in x] # flatten list
    # print(sentences)

    # remove punctuations, numbers and special characters
    clean_sentences = pd.Series(sentences).str.replace("[^a-z A-Z 0-9]", " ")

    # make alphabets lowercase
    clean_sentences = [s.lower() for s in clean_sentences]
    #print(clean_sentences)


    # remove stopwords from the sentences
    clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]
    #print(clean_sentences)

    sentence_vectors = sentence_vector_func(clean_sentences)

    # similarity matrix
    sim_mat = np.zeros([len(sentences), len(sentences)])
    #print(sim_mat)

    # Finding the similarities between the sentences
    for i in range(len(sentences)):
        for j in range(len(sentences)):
            if i != j:
                sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]


    nx_graph = nx.from_numpy_array(sim_mat)
    scores = nx.pagerank(nx_graph)
    #print(scores)

    ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)))
    # Extract sentences as the summary
    summarised_string = ''
    for i in range(n):

        try:
            summarised_string = summarised_string + str(ranked_sentences[i][1])
        except IndexError:
            print ("Summary Not Available")

    return (summarised_string)

In [ ]:
import nltk
nltk.download('punkt_tab')
nltk.download('punkt')
nltk.download('stopwords')

print("Kindly let me know in how many sentences you want the summary - ")
x = int(input())

summary_dictionary = {}

for key in text_dictionary:

    para = text_dictionary[key]
    print("Summary of the article - ",key)
    summary = summary_text(para,x)
    summary_dictionary[key] = summary

    print(summary)
    print('='*120)

print ("*"*40,"The process has been completed successfully","*"*40)

In [23]:
summary_table = pd.DataFrame(list(summary_dictionary.items()),columns = ['TEST DATASET','Summary'])

In [24]:
data_table = pd.DataFrame(list(text_dictionary.items()),columns = ['TEST DATASET','Introduction'])

In [25]:
# Combining the findings into the table
result  = pd.concat([data_table , summary_table['Summary']], axis = 1 , sort = False)
result

,TEST DATASET,Introduction,Summary
0,1,Acnesol Gel is an antibiotic that fights bacte...,Acnesol Gel is an antibiotic that fights bacte...
1,2,Ambrodil Syrup is used for treating various re...,It works by thinning and loosens mucus in the ...
2,3,Augmentin 625 Duo Tablet is a penicillin-type ...,It is used to treat infections of the lungs (e...
3,4,Azithral 500 Tablet is an antibiotic used to t...,These are usually temporary and subside with t...
4,5,Alkasol Oral Solution is a medicine used in th...,Alkasol Oral Solution is a medicine used in th...
...,...,...,...
995,996,Azapure Tablet belongs to a group of medicines...,NaN
996,997,Arimidex 1mg Tablet is used alone or with oth...,NaN
997,998,Arpimune ME 100mg Capsule is used to prevent y...,NaN
998,999,Amlodac CH Tablet is a combination medicine us...,NaN


In [ ]:
# Saving it to a file (remove the '#' to save)
#result.to_csv("Summary_File.csv")

In [6]:
import nltk
import numpy as np
import pandas as pd
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx
from sklearn.feature_extraction.text import TfidfVectorizer

# Load GloVe embeddings (make sure the path is correct)
word_embeddings = {}
with open('/content/glove.6B.200d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        word_embeddings[word] = coefs

# Remove stopwords from sentences
def remove_stopwords(sen):
    stop_words = stopwords.words('english')
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

# Compute sentence vectors using GloVe embeddings
def sentence_vector_func(sentences_cleaned):
    sentence_vector = []
    for i in sentences_cleaned:
        if len(i) != 0:
            v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
        else:
            v = np.zeros((100,))
        sentence_vector.append(v)
    return sentence_vector

# Generate summary using TF-IDF and PageRank
def summary_text_with_tfidf(test_text, n=5):
    sentences = sent_tokenize(test_text)

    # Clean sentences and remove stopwords
    clean_sentences = pd.Series(sentences).str.replace("[^a-z A-Z 0-9]", " ")
    clean_sentences = [s.lower() for s in clean_sentences]

    # TF-IDF Vectorizer to get sentence relevance scores
    tfidf_vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf_vectorizer.fit_transform(clean_sentences)

    # Compute cosine similarities
    sim_mat = cosine_similarity(tfidf_matrix, tfidf_matrix)

    # Create similarity graph
    nx_graph = nx.from_numpy_array(sim_mat)

    # Use PageRank algorithm to rank sentences based on similarity
    scores = nx.pagerank(nx_graph)

    # Rank sentences by their PageRank scores
    ranked_sentences = sorted(((scores[i], s) for i, s in enumerate(sentences)), reverse=True)

    # Construct the summary based on the top 'n' sentences
    summarised_string = ""
    for i in range(n):  # Top 'n' sentences
        summarised_string += ranked_sentences[i][1] + " "

    return summarised_string

# Example usage (you can put this in a loop or function to handle multiple texts)
text = """The Mughal Empire, one of the most powerful and enduring empires in Indian history, was founded by Babur in 1526 after his victory at the Battle of Panipat against the Sultan of Delhi, Ibrahim Lodi. Babur, a descendant of Timur and Genghis Khan, established the Mughal dynasty, which ruled large parts of the Indian subcontinent for several centuries. His son, Humayun, faced significant challenges, including battles for succession and exile, but regained the throne with the help of the Persian Empire.

Akbar, Humayun’s son, is often regarded as the greatest Mughal emperor, known for his administrative reforms, religious tolerance, and expansion of the empire. Akbar’s policies integrated various cultures, languages, and religions, leading to a flourishing of art, architecture, and commerce. The empire continued to prosper under his successors, Jahangir, Shah Jahan, and Aurangzeb, each contributing to the cultural richness and territorial expansion of the empire. Shah Jahan is best known for commissioning the Taj Mahal, a symbol of Mughal architectural excellence.

However, after Aurangzeb’s death in 1707, the empire began to decline due to internal strife, external invasions, and weakening central authority. The British East India Company capitalized on this decline, eventually leading to British colonial rule in India. The Mughal Empire formally came to an end in 1857 after the Indian Rebellion, when the last Mughal emperor, Bahadur Shah Zafar, was exiled.
"""
# Sample text

# Call the summary function
full_summary = summary_text_with_tfidf(text, n=5)  # Passing the number of sentences for the summary
print("Full Text Summary:")
print(full_summary)


KeyboardInterrupt: 